In [1]:
import pyodbc
import random
import string
import datetime
from faker import Faker

In [2]:
# Connect to the database
server = '(local)\\UCNSOFTWARE'
database = 'gtl_database'
conn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;')
cursor = conn.cursor()

In [ ]:
# Init faker
fake = Faker()
Faker.seed(0)

In [ ]:
# Generate and insert data into item_type table
item_types = [('Book', True, 'BOOK'),
              ('Rare Book', False, 'BOOK'),
              ('Reference Book', False, 'BOOK'),
              ('Map', False, 'MAP'),
              ('DVD', True, 'MISC'),
              ('CD', True, 'MISC'),
              ('Magazine', True, 'MISC')]
for item_type in item_types:
    query = "INSERT INTO item_type ([type], [loanable], [item_type]) VALUES (?, ?, ?)"
    cursor.execute(query, item_type)
cursor.commit()

# Generate and insert data into subject table
subjects = ['Science', 'History', 'Art', 'Music', 'Sports']
for subject in subjects:
    query = "INSERT INTO subject ([subject]) VALUES (?)"
    cursor.execute(query, subject)
cursor.commit()

# Generate and insert data into author table
for i in range(1, 1001):
    first_name = fake.first_name()
    last_name = fake.last_name()
    query = "INSERT INTO author ([id], [first_name], [last_name]) VALUES (?, ?, ?)"
    cursor.execute(query, i, first_name, last_name)
cursor.commit()

# Generate and insert data into item table
for i in range(1, 10001):
    title = fake.text(max_nb_chars=40)
    date = datetime.datetime.now()
    for_acquisition = random.randint(0, 1)
    description = fake.paragraph()
    item_type = random.choice(item_types)
    isbn = None if item_type[2] != 'BOOK' else fake.isbn13()
    area = None if item_type[2] != 'MAP' else fake.location_on_land()[2]
    misc_identifier = None if item_type[2] != 'MISC' else  fake.sbn9()
    query = "INSERT INTO item ([id], [title], [date], [for_acquisition], [description], [type], [isbn], [area], [misc_identifier]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cursor.execute(query, i, title, date, for_acquisition, description, item_type[0], isbn, area, misc_identifier)
cursor.commit()

# Generate and insert data into item_subjects table
for i in range(1, 10001):
    num_subjects = random.randint(1, 3)
    selected_subjects = random.sample(subjects, num_subjects)
    for subject in selected_subjects:
        query = "INSERT INTO item_subjects ([subject], [I_id]) VALUES (?, ?)"
        cursor.execute(query, subject, i)
cursor.commit()

# Generate and insert data into item_authors table
for i in range(1, 10001):
    num_authors = random.randint(1, 3)
    selected_authors = random.sample(range(1, 1001), num_authors)
    for author in selected_authors:
        query = "INSERT INTO item_authors ([I_id], [A_id]) VALUES (?, ?)"
        cursor.execute(query, i, author)
cursor.commit()

# Generate and insert data into item_copy table
for i in range(1, 10001):
    barcode = 1000000 + i
    destroyed = 1 if random.random() < 0.05 else 0
    item_id = random.randint(1, 10000)
    query = "INSERT INTO item_copy ([barcode], [destroyed], [I_id]) VALUES (?, ?, ?)"
    cursor.execute(query, barcode, destroyed, item_id)
cursor.commit()

# Generate and insert data into person table
for i in range(1, 10001):
    ssn = 100000000 + i
    first_name = fake.first_name()
    last_name = fake.last_name()
    birth_date = datetime.datetime.now() - datetime.timedelta(days=random.randint(365*20, 365*60))
    sex = random.choice(['M', 'F'])
    campus = random.choice(['Campus A', 'Campus B', 'Campus C'])
    phone_no = ''.join(random.choice(string.digits) for _ in range(10))
    address = fake.address()
    l_flag = 1 if random.random() < 0.05 else 0
    role = None if l_flag == 0 else random.choice(['Chief librarian', 'Departmental', 'Associate librarian', 'Reference librarian', 'check-out librarian', 'Library assistant'])
    m_flag = 1 if l_flag == 0 else random.randint(0, 1)
    member_type = None if m_flag == 0 else random.choice(['Regular', 'Professor'])
    query = "INSERT INTO person ([ssn], [first_name], [last_name], [birth_date], [sex], [campus], [phone_no], [address], [l_flag], [role], [m_flag], [type]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cursor.execute(query, ssn, first_name, last_name, birth_date, sex, campus, phone_no, address, l_flag, role, m_flag, member_type)
cursor.commit()

# Generate and insert data into loan table
for i in range(1, 10001):
    barcode = 1000000 + i
    member_ssn = random.randint(100000001, 100010000) #existing ssn
    reservation_date = datetime.datetime.now() + datetime.timedelta(days=random.randint(-365, 30))
    start_date = None if reservation_date > datetime.datetime.now() else reservation_date + datetime.timedelta(days=random.randint(1, 7))
    returned_date = None if start_date is None else start_date + datetime.timedelta(days=random.randint(1, 30))
    returned_date = None if returned_date == None or returned_date > datetime.datetime.now() or random.random() < 0.05 else returned_date #lost items
    notice_sent = random.randint(0, 1)
    query = "INSERT INTO loan ([Id], [barcode], [member_ssn], [reservation_date], [start_date], [returned_date], [notice_sent]) VALUES (?, ?, ?, ?, ?, ?, ?)"
    cursor.execute(query, i, barcode, member_ssn, reservation_date, start_date, returned_date, notice_sent)
cursor.commit()

# Generate and insert data into member_card table
for i in range(1, 5001):
    card_number = 100000 + i
    issued = datetime.datetime.now() - datetime.timedelta(days=random.randint(0, 365*5))
    member_ssn = random.randint(100000001, 100010000) #existing ssn
    query = "INSERT INTO member_card ([card_number], [issued], [member_ssn]) VALUES (?, ?, ?)"
    cursor.execute(query, card_number, issued, member_ssn)
cursor.commit()

In [3]:
conn.close()

In [ ]:
# conn.rollback()